In [60]:
import advent
advent.scrape(2024, 9)
data = advent.get_lines(9)[0]

data = '2333133121414131402'

memory = []

for i, c in enumerate(data):
    if i % 2 == 0: memory.extend([i // 2] * int(c))
    else: memory.extend([-1] * int(c))

In [61]:
def part1(memory):
    ix, revix = 0, len(memory) - 1
    result = 0
    # Traverse through the list from both ends, swapping -1s with numbers
    # Stop until the two ix'es reach each other
    while ix <= revix:
        if memory[ix] > -1:
            result += (memory[ix] * ix)
            ix += 1
            continue
        elif memory[revix] == -1:
            revix -= 1
            continue
        memory[ix], memory[revix] = memory[revix], memory[ix]
        revix -= 1

    # At the end, we keep traversing until we encounter a single -1
    # This is because ix (and the checksum) hasnt seen every number yet
    while memory[ix] > -1:
        result += (memory[ix] * ix)
        ix += 1
    return result, memory

print(part1(memory.copy())[0])


1928


In [67]:
# Part 2
# My idea is to keep a list of gaps, which are tuples (start, length).
# Starting at the end, we find the first suitable gap for a range (length check), and update the gap length (0 if needed)
def find_fit(gaps, length):
    for i, gap in enumerate(gaps):
        if gap[1] >= length:
            #gaps[i] = (gap[0], gap[1] - length)
            return gap, i
    return None, None # No fit

def update_memory(memory, gaps, start, length):
    # Given the start of a memory range, update memory and gaps by moving that range forward to the first available gap
    content = memory[start]
    gap, i = find_fit(gaps, length)
    if gap is None or gap[0] > start: return memory, gaps # No update needed
    # Update gaps
    gaps[i] = (gap[0] + length, gap[1] - length)
    # Update memory
    memory[start:(start + length)] = [-1] * length
    memory[gap[0]:(gap[0] + length)] = [content] * length
    return memory, gaps

def get_length(memory, start):
    # given start ix, find the length of the range
    content = memory[start]
    length = 1
    while start + length < len(memory) and memory[start + length] == content:
        length += 1
    return length

ix = 0
gaps = []
for i, c in enumerate(data):
    if i % 2 == 0: ix += int(c)
    if i % 2 == 1:
        gaps.append((ix, int(c)))
        ix += int(c)

memory_p2 = memory.copy()

ix = len(memory_p2) - 1
while ix > 0:
    # We only want to swap starts of ranges, so memory[ix-1] must be different
    if memory_p2[ix] == -1 or memory_p2[ix] == memory_p2[ix-1]:
        ix -= 1
        continue
    start, length = ix, get_length(memory_p2, ix)
    memory_p2, gaps = update_memory(memory_p2, gaps, start, length)
    ix -= 1

def checksum(memory):
    result = 0
    for i, c in enumerate(memory):
        if c == -1: continue
        result += i * memory[i]
    return result

print(checksum(memory_p2))

2858


In [ ]:
# Ok this is probably overkill...
from typing import NamedTuple

def triang(n):
    return n * (n + 1) // 2

class MemoryRange(NamedTuple):
    start: int
    length: int
    content: int

    def is_empty(self): return self.content == -1
    def checksum(self):
        # content * (start + start + 1 + start + 2 + ... + start + length - 1)
        if self.content == -1: return 0
        return self.content * ((self.length - 1) * self.start + triang(self.length - 1))

    def fill(self, other: 'MemoryRange'):
        # Fill this (empty) range with something else, return new range(s)
        # Or none if that is impossible
        if not self.is_empty(): return None
        if self.length > other.length: return None 
        return [
            MemoryRange(self.start, other.length, other.content),
            MemoryRange(self.start + other.length, self.length - other.length, self.content),

        ]

assert MemoryRange(0, 3, 2).checksum() == 6
assert MemoryRange(10, 2, 5).checksum() == 55